In [2]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
path1 = r"C:\Users\abebe\Documents\George Mason University\6- SUMME 2023\Team-SkySci-Analytics\Metadata Merge Transform\ground_weather_merged.csv"
path2 = r"C:\Users\abebe\Documents\George Mason University\6- SUMME 2023\Team-SkySci-Analytics\Atmospheric Data Scraper\atmospheric_data_dewPt_master.csv"

In [ ]:
atmosphericData = pd.read_csv(path2)
groundWeatehrData = pd.read_csv(path1)
atmosphericData.head()
# pressure is in units of pascal, which we will convert to millibars

In [ ]:
# Converting timestamp format 
# atmosphericData['Timestamp'] = pd.to_datetime(atmosphericData['Timestamp'], format='%m/%d/%Y %H:%M').dt.strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
# # Group the DataFrame by longitude and latitude
# grouped_data = atmosphericData.groupby(['Longitude', 'Latitude'])

# # Iterate over the groups
# for group, group_df in grouped_data:
# # Print longitude, latitude, and the corresponding data for each group to look at all the available coordinates
#     print("Longitude:", group[0])
#     print("Latitude:", group[1])
#     print("Grouped data:")
#     print(group_df)
#     print()

In [ ]:
#selected area of coordinates 
long = -77.02804541
lat= 38.53080152

In [ ]:
# # Checking if there are datapoint within the selected alt and long coordinates 
# Atm_data = atmosphericData[(atmosphericData['Latitude'] == lat) & (atmosphericData['Longitude'] == long)]
# if len(df) > 0:
#     print("There are rows with the specified latitude and longitude.")
# else:
#     print("There are no rows with the specified latitude and longitude.")

In [ ]:
# df.to_csv("dewPt_data.csv", index=False)

# Relative Humidity Calculation

In [4]:
path3 = r"C:\Users\abebe\Documents\George Mason University\6- SUMME 2023\Team-SkySci-Analytics\Atmospheric Data Scraper\clean_atmospheric_data.csv"
clean_data= pd.read_csv(path3)

The following resources are used to calculate RH and Saturated VP and Actual Vp

* https://www.weather.gov/media/epz/wxcalc/tempConvert.pdf  <<- Temp Convertion 
* https://www.weather.gov/media/epz/wxcalc/vaporPressure.pdf <<- RH AND pressure Calculation
* https://andrewsforest.oregonstate.edu/sites/default/files/lter/data/studies/ms01/dewpt_vpd_calculations.pdf


**RH = (e / es) * 100**

Where:

- RH is the relative humidity expressed as a percentage
- e is the actual water vapor pressure
- es is the saturation vapor pressure at the same temperature.**

The saturation vapor pressure (es) at a given temperature can be canculated using this formuala :


**e =  6.11 * 10^((7.5 * Td) / (237.3 + Td))** 


**es = 6.11 * 10^((7.5 * T) / (237.3 + T))**



In [5]:
# Converting atmospheric temperature and dew point temperature to degrees Celsius
clean_data['TMP_C'] = clean_data['Temperature'] - 237.16
clean_data['dewPT_C'] = clean_data['Dewpt'] - 237.16
round(max(clean_data.dewPT_C),2)

59.37

In [6]:
# Calculate saturation vapor pressure over water
clean_data.loc[:, 'Saturated_Vapor_Pressur'] = 6.11 * 10 ** ((7.5 * clean_data['TMP_C']) / (237.3 + clean_data['TMP_C']))
# Calculate actual vapor pressure
clean_data.loc[:, 'Actual_Vapor_Pressure'] = 6.11 * 10 ** ((7.5 * clean_data['dewPT_C']) / (237.3 + clean_data['dewPT_C']))
clean_data.head()
# # Calculate relative humidity
clean_data.loc[:, 'RH_water'] = (clean_data['Actual_Vapor_Pressure'] / clean_data['Saturated_Vapor_Pressur']) * 100
clean_data.head(2)

,Timestamp,Level,Temperature,Humidity,altitude (ft),Dewpt,TMP_C,dewPT_C,Saturated_Vapor_Pressur,Actual_Vapor_Pressure,RH_water
0,2022-09-01 00:00:00,200.0,217.590439,22.580428,38615.046,288.822052,-19.569561,51.662052,1.294026,133.945452,10351.06451
1,2022-09-01 00:00:00,225.0,221.009056,23.895230,36195.497,288.822052,-16.150944,51.662052,1.731018,133.945452,7737.95657


In [9]:
a_water = 17.2693882
a_ice = 21.8745584
b_water = 35.86
b_ice = 7.66
TMP = clean_data['TMP_C']
# TMP = df['TMP_C']  # Temperature in Celsius
RH_water = clean_data['RH_water']  # Relative humidity with respect to water

clean_data.loc[: ,'RH_ice'] = RH_water * np.exp(a_water * (TMP - 273.16) / (TMP - b_water) - a_ice * (TMP - 273.16) / (TMP - b_ice))
clean_data
# df.head()

C:\Users\abebe\anaconda3\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)


,Timestamp,Level,Temperature,Humidity,altitude (ft),Dewpt,TMP_C,dewPT_C,Saturated_Vapor_Pressur,Actual_Vapor_Pressure,RH_water,RH_ice
0,2022-09-01 00:00:00,200.0,217.590439,22.580428,38615.046,288.822052,-19.569561,51.662052,1.294026,133.945452,10351.064510,3.120283e-59
1,2022-09-01 00:00:00,225.0,221.009056,23.895230,36195.497,288.822052,-16.150944,51.662052,1.731018,133.945452,7737.956570,1.510790e-70
2,2022-09-01 00:00:00,250.0,226.554466,19.933957,33984.704,288.822052,-10.605534,51.662052,2.723741,133.945452,4917.701050,8.061241e-99
3,2022-09-01 00:00:00,275.0,231.823334,17.869399,31946.256,288.822052,-5.336666,51.662052,4.106695,133.945452,3261.636587,4.423322e-150
4,2022-09-01 00:00:00,300.0,236.828835,14.600191,30052.740,288.822052,-0.331165,51.662052,5.964306,133.945452,2245.784392,7.905486e-266
...,...,...,...,...,...,...,...,...,...,...,...,...
34627,2022-12-22 23:00:00,400.0,242.988808,46.421067,23564.347,285.333466,5.828808,48.173466,9.243733,112.631907,1218.467822,0.000000e+00
34628,2022-12-22 23:00:00,425.0,246.693436,45.005139,22151.114,285.333466,9.533436,48.173466,11.904560,112.631907,946.124076,inf
34629,2022-12-22 23:00:00,450.0,250.197746,46.562771,20803.669,285.333466,13.037746,48.173466,15.019169,112.631907,749.921040,inf
34630,2022-12-22 23:00:00,475.0,253.413284,51.586541,19515.537,285.333466,16.253284,48.173466,18.484451,112.631907,609.333255,inf


In [ ]:
# print( round(min(df['RH_water']),2))
# print( round(max(df['RH_water']),2))

### Merging Atmospheric Data with Skydata

In [ ]:
Atm_data = df.groupby('Timestamp').mean().reset_index().rename(columns={'Timestamp': 'Datetime', 'Dew Point Temperature' : 'atm_dewPt'})
# Strip the last date from the atmospheric data to ensure consistency with the ground surface weather data

Atm_data['Datetime'] = Atm_data['Datetime'].astype(str)
Atm_data = Atm_data[~Atm_data['Datetime'].str.contains('2022-12-23')]


# print ("Ground Surface Data")
grd_data = groundWeatehrData.groupby('Datetime').mean('Temperature')


In [ ]:
# Strip the last date from the atmospheric data to ensure consistency with the ground surface weather data
Atm_data = Atm_data[~Atm_data['Datetime'].str.contains('2022-12-23')]
print ("RAP Atmospheric Data")
print(Atm_data.shape)

print ("Ground Surface Data")
grd_data = groundWeatehrData.groupby('Datetime').mean('Temperature').reset_index()
print(grd_data.shape)

In [ ]:
# combined_data = pd.merge(Atm_data, grd_data, on="Datetime")
# #Renaming Ground Temprature and atmoshpheric temprature 
# # combined_data.rename(columns={'Temperature_x': 'atm_TMP', 'Temperature_y': 'grd_TMP'}, inplace=True)


In [ ]:
df.to_csv('ConditionedAtmosphericData.csv', index=False)

In [ ]:
# Convert the datetime column to Timestamp format
# combined_data.reset_index(inplace=True)

# combined_data['Datetime']= pd.to_datetime(combined_data['Datetime'], format='%Y-%m-%d %H:%M:%S')
# combined_data['Datetime'].dtypes

In [ ]:
combined_data.loc[:, 'atm_TMP(F)'] = combined_data['atm_TMP'] - 237.16

In [ ]:
combined_data.head(4)

## Data Exploration and Visualization

In [ ]:
# combined_data.set_index('Datetime', inplace=True
import plotly.express as px
combined_data.head(2)

In [ ]:
# Plot the temperature data for the entire dataset
plt.figure(figsize=(15, 7))
sns.set_style('darkgrid')
sns.lineplot(data=combined_data, x='Datetime', y='atm_TMP(F)', color='darkblue')

# plt.xlabel('Date', fontsize=13, fontweight='bold')
plt.ylabel('Temperature (°F)', fontsize=13, fontweight='bold')
plt.xticks(rotation=30, fontsize=12)
plt.yticks(fontsize=12)
plt.title('Hourly Atmospheric Temperature (Sept 01 - Dec 22)', fontsize=16, fontweight='400', fontfamily='Arial')

plt.show()

In [ ]:
# avg_TMP = combined_data['atm_TMP'].mean()

# Plot the temperature data for the entire dataset
plt.figure(figsize=(15, 7))
sns.set_style('darkgrid')
sns.lineplot(data=combined_data['Pressure'], color='darkblue')
# Add the average line
# plt.axhline(avg_TMP, color='red', linestyle='--', label='Average', lw='3')

plt.xlabel('Date', fontsize=13, fontweight='bold')
plt.ylabel('Temperature (°F)', fontsize=13, fontweight='bold')
plt.xticks(rotation=30, fontsize=12)
plt.yticks(fontsize=12)
plt.title('Hourly Atmospheric Temperature ((Sept 01 - Dec 22) ', fontsize=16, fontweight='400', fontfamily='Arial')

plt.show()

In [ ]:
plt.figure(figsize=(15, 7))
sns.set_style('darkgrid')
sns.lineplot(data=combined_data['Humidity'], color='darkgreen')
plt.xlabel('Datetime', fontsize=13, fontweight='bold')
plt.ylabel('Humidity (%)', fontsize=13, fontweight='bold')
plt.xticks(rotation=30, fontsize=12)
plt.yticks(fontsize=12)
plt.title('Hourly Atmospheric Humidity (Sept 01 - Dec 22)',  fontsize=16, fontweight='400', fontfamily='Arial')

plt.show()

In [ ]:
# combined_data.set_index('Datetime', inplace=True)
# combined_data.head(3)

In [ ]:
combined_data.index = pd.to_datetime(combined_data.index)
combined_data.head()
# Create a subset for 4-week span with midnight data
start_date = pd.Timestamp('2022-09-01 00:00:00')
end_date = start_date + pd.DateOffset(weeks=4)
midnight_subset = combined_data.between_time('00:00:00', '00:00:00').loc[start_date:end_date]
morning_subset = combined_data.between_time('06:00', '06:00').loc[start_date:end_date]
afternoon_subset = combined_data.between_time('12:00', '12:00').loc[start_date:end_date]
evening_subset = combined_data.between_time('18:00', '18:00').loc[start_date:end_date]


In [ ]:
# Plot the temperature data
plt.figure(figsize=(15, 7))
sns.set_style('darkgrid')
sns.lineplot(data=midnight_subset['atm_TMP(F)'], color='darkblue', ls='dashed', marker='s')

plt.xlabel('Date', fontsize=13, fontweight='bold')
plt.ylabel('Temperature (°F)', fontsize=13, fontweight='bold')
plt.xticks(rotation=30, fontsize=12)
plt.yticks(fontsize=12)
plt.title('Midnight Atmospheric Temperature over 4 Weeks Span', fontsize=16, fontweight='400', fontfamily='Arial')

plt.show()

In [ ]:
# Plot the temperature data
plt.figure(figsize=(15, 7))
sns.set_style('darkgrid')
sns.lineplot(data=midnight_subset['Pressure'], color='purple', ls='dashed', marker='s')

plt.xlabel('Date', fontsize=13, fontweight='bold')
plt.ylabel('hPa', fontsize=13, fontweight='bold')
plt.xticks(rotation=30, fontsize=12)
plt.yticks(fontsize=12)
plt.title('Midnight Atmospheric Pressure over 4 Weeks Span', fontsize=16, fontweight='400', fontfamily='Arial')

plt.show()

In [ ]:
combined_data.index = pd.to_datetime(combined_data.index)
#create a subset for 4 week spans 
start_date = pd.Timestamp('2022-09-01 00:00:00')
end_date = start_date + pd.DateOffset(weeks=4)
subset = combined_data.between_time('00:00', '23:00').loc[start_date:end_date]

#plot
plt.figure(figsize=(15, 7))
sns.set_style('darkgrid')
sns.lineplot(data=subset['atm_TMP'], color='darkgreen', ls='dashed', marker='')

plt.xlabel('Date', fontsize=13, fontweight='bold')
plt.ylabel('Temperature (°F)', fontsize=13, fontweight='bold')
plt.xticks(rotation=30, fontsize=12)
plt.yticks(fontsize=12)
plt.title(' Atmospheric Temperature over 4 Weeks Span', fontsize=16, fontweight='400', fontfamily='Arial')

In [ ]:
combined_data.index = pd.to_datetime(combined_data.index)
#create a subset for 4 week spans 
start_date = pd.Timestamp('2022-09-01 00:00:00')
end_date = start_date + pd.DateOffset(weeks=4)
subset = combined_data.between_time('00:00', '23:00').loc[start_date:end_date]

#plot
plt.figure(figsize=(15, 7))
sns.set_style('darkgrid')
sns.lineplot(data=subset['Pressure'], color='darkgreen', ls='dashed', marker='')

plt.xlabel('Date', fontsize=13, fontweight='bold')
plt.ylabel('hPa', fontsize=13, fontweight='bold')
plt.xticks(rotation=30, fontsize=12)
plt.yticks(fontsize=12)
plt.title(' Atmospheric Pressure over 4 Weeks Span', fontsize=16, fontweight='400', fontfamily='Arial')

In [ ]:
# Plot the Humidity data
plt.figure(figsize=(15, 7))
sns.set_style('darkgrid')
sns.lineplot(data=midnight_subset['Humidity'], color='darkblue', ls='dashed', marker='')

plt.xlabel('Date', fontsize=13, fontweight='bold')
plt.ylabel('Humidity (%)', fontsize=13, fontweight='bold')
plt.xticks(rotation=30, fontsize=12)
plt.yticks(fontsize=12)
plt.title(' Midnight Atmospheric Humidity over 4 Weeks Span', fontsize=16, fontweight='400', fontfamily='Arial')


plt.show()

In [ ]:
# Plot the temperature data
plt.figure(figsize=(15, 7))
sns.set_style('darkgrid')
sns.lineplot(data=midnight_subset['atm_TMP'], color='darkblue')

sns.lineplot(data=morning_subset['atm_TMP'], color='purple', label='Morning', linestyle='solid' )
sns.lineplot(data=afternoon_subset['atm_TMP'], color='darkgreen', label='Noon', linestyle='solid', lw='1.7')
sns.lineplot(data=evening_subset['atm_TMP'], color='black', label='Evening', lw='1')
plt.axhline(avg_TMP, color='red', linestyle='--', label='Average', lw='2'
           )

plt.xlabel('Date', fontsize=13, fontweight='bold')
plt.ylabel('Temperature (°F)', fontsize=13, fontweight='bold')
plt.xticks(rotation=30, fontsize=12)
plt.yticks(fontsize=12)
plt.title('Atmospheric Temperature over 4 Weeks Span Every 6 Hours', fontsize=16, fontweight='400', fontfamily='Arial')

plt.show()